In [6]:
#Author: Lalitha Gorantla
#source: chat GPT
#11/29/23

import pandas as pd
# Load data into pandas DataFrame
df = pd.read_csv('GDBCup - Week3.csv')

In [7]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Select features and target
features = df[['Open', 'High', 'Low', 'Volume']].values
labels = df['Close'].values

# Normalize features
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_scaled, labels, test_size=0.2, random_state=42)

# Build a simple neural network model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Save the model
model.save('NN_Model_Unresticted.h5')


Epoch 1/10
28/28 [==============================] - 1s 2ms/step - loss: 20914492.0000
Epoch 2/10
28/28 [==============================] - 0s 2ms/step - loss: 20904272.0000
Epoch 3/10
28/28 [==============================] - 0s 2ms/step - loss: 20878838.0000
Epoch 4/10
28/28 [==============================] - 0s 2ms/step - loss: 20825492.0000
Epoch 5/10
28/28 [==============================] - 0s 2ms/step - loss: 20730116.0000
Epoch 6/10
28/28 [==============================] - 0s 2ms/step - loss: 20575598.0000
Epoch 7/10
28/28 [==============================] - 0s 2ms/step - loss: 20347650.0000
Epoch 8/10
28/28 [==============================] - 0s 2ms/step - loss: 20032058.0000
Epoch 9/10
28/28 [==============================] - 0s 2ms/step - loss: 19616986.0000
Epoch 10/10
28/28 [==============================] - 0s 2ms/step - loss: 19098472.0000


In [8]:
from tensorflow.keras.models import load_model

# Load the model
model = load_model('NN_Model_Unresticted.h5')  # Ensure this is the correct path where the model is saved
model

In [9]:
#Author: Lalitha Gorantla
#source: chat GPT
#11/15/23

#The program implements a trading strategy using Backtrader, a Python library for backtesting trading algorithms. 
#It employs a combination of Simple Moving Average (SMA) crossovers, Commodity Channel Index (CCI) divergences, 
#and a dynamic stop loss/take profit system based on the Average True Range (ATR) to make trading decisions, 
#with the ability to plot divergence markers and manage multiple trade orders.
from datetime import datetime
import backtrader as bt
import pandas as pd
import numpy as np

def check_market_hours(time):
    """Checks if the current time is within market hours."""
    hour, minute = time.hour, time.minute
    if (9 <= hour < 14) or (hour == 8 and minute > 29) or (hour == 14 and minute < 40):
        return True
    return False

class DivergenceMarker(bt.Indicator):
    """Custom indicator to plot bullish and bearish divergence markers on a chart."""
    lines = ('bullish', 'bearish',)
    plotinfo = dict(subplot=True)
    
    plotlines = dict(
        bullish=dict(marker='^', markersize=8.0, color='green', fillstyle='full'),
        bearish=dict(marker='v', markersize=8.0, color='red', fillstyle='full')
    )

class AnchoredVWAP(bt.Indicator):
    lines = ('vwap_high', 'vwap_low')  # Added line for vwap_low
    params = (('start_bar', 0),)
    
    def __init__(self):
        self.addminperiod(self.params.start_bar + 1)
        self.bar_volume = self.data.volume.get(size=self.params.start_bar)
        self.bar_high = self.data.high.get(size=self.params.start_bar)  # Using high instead of close
        self.bar_low = self.data.low.get(size=self.params.start_bar)  # Added for vwap_low

    def next(self):
        data_len = len(self.bar_volume)
        max_index = min(data_len, self.p.start_bar + 1)

        cum_vol = sum(self.bar_volume[-i] for i in range(max_index))
        cum_vol_price_high = sum(self.bar_volume[-i] * self.bar_high[-i] for i in range(max_index))
        cum_vol_price_low = sum(self.bar_volume[-i] * self.bar_low[-i] for i in range(max_index))  # Added for vwap_low

        self.lines.vwap_high[0] = cum_vol_price_high / cum_vol if cum_vol else float('nan')  # Using vwap_high
        self.lines.vwap_low[0] = cum_vol_price_low / cum_vol if cum_vol else float('nan')  # Added for vwap_low


class CombinedStrategy(bt.Strategy):
    """Combines SMA crossover and CCI divergence detection for trading decisions."""
    #default parameters
    params = dict(
        cci_period=3,
        lookback=20,
        fast_sma_period=2,
        slow_sma_period=4,
        vwap_start_bar=0,  # Number of bars before regular trading hours to start anchoring VWAP
        atr_period=14,  # ATR period for market volatility
        atr_multiplier=1.5  # Multiplier for determining stop loss/take profit based on ATR
        
    )
    
    def __init__(self):
        # stop-loss and take-profit
        self.buy_order = None
        self.sell_order = None
        self.order = None
        
        # Load the pre-trained neural network model
        self.model = load_model('NN_Model_Unresticted.h5')
        
        # Initialize a list to store trade data
        self.trade_list = []

        # Additional attributes to store trade data
        self.current_trade = {}
        
        # Indicators for SMA crossover strategy
        self.fast_sma = bt.indicators.SimpleMovingAverage(period=self.params.fast_sma_period)
        self.slow_sma = bt.indicators.SimpleMovingAverage(period=self.params.slow_sma_period)
        self.crossover = bt.ind.CrossOver(self.fast_sma, self.slow_sma)
        
        # Indicator for anchored VWAP
        self.vwap = AnchoredVWAP(start_bar=self.params.vwap_start_bar)
        
        # ATR Indicator for determining stop loss and take profit dynamically
        self.atr = bt.indicators.AverageTrueRange(period=self.params.atr_period)

        # Indicators for CCI divergence strategy
        self.cci = bt.indicators.CCI(self.data, period=self.params.cci_period)
        self.ohlc_peaks = []  # Store peaks of the OHLC data
        self.ohlc_peak_indexes = []  # Store indexes of the OHLC peaks
        self.cci_peaks = []  # Store peaks of the CCI data
        self.cci_peak_indexes = []  # Store indexes of the CCI peaks
        
        # Store valleys of the OHLC and CCI data
        self.ohlc_valleys = []
        self.ohlc_valley_indexes = []
        self.cci_valleys = []
        self.cci_valley_indexes = []    
        
        # Create an instance of the divergence marker for plotting
        self.div_marker = DivergenceMarker()
    
    def log(self, txt, dt=None):
        ''' Logging function fit this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
    
    def notify_trade(self, trade):
        if trade.justopened:
            self.current_trade = {
                'Trade': 'Long' if trade.size > 0 else 'Short',
                'Date': self.data.datetime.datetime(0).strftime('%m/%d/%Y %I:%M:%S %p'),
                'Price': trade.price,
                'Contracts': abs(trade.size),
                'Position Value': trade.size * trade.price
            }
        elif trade.isclosed:            
            self.current_trade.update({
                'Exit Date': self.data.datetime.datetime(0).strftime('%m/%d/%Y %I:%M:%S %p'),
                'Exit Price': trade.price,
                'Profit': trade.pnl,
                '% Profit': "{:.2f}%".format((trade.pnl / self.current_trade['Price']) * 100),
                '% Change': "{:.2f}%".format(((trade.price / self.current_trade['Price']) - 1) * 100),
                'Cumulative Profit': sum(t['Profit'] for t in self.trade_list) + trade.pnl
            })
            self.trade_list.append(self.current_trade)
            self.current_trade = {}

    def stop(self):
        # Convert the trade list to a DataFrame
        trades_df = pd.DataFrame(self.trade_list)

        # Saving the DataFrame to a CSV file
        trades_df.to_csv('backtrader_Unrestricted_output.csv', index=False)
        print("Backtrader output saved to 'backtrader_Unrestricted_output.csv'.")        

    def get_nn_prediction(self):
        # Prepare the current data for the model
        current_features = np.array([[
            self.datas[0].open[0], 
            self.datas[0].high[0], 
            self.datas[0].low[0],
            self.datas[0].volume[0]
        ]])

        # Normalize features as done during training
        current_features = scaler.transform(current_features)  # Use the same scaler as in training

        # Use model to make predictions
        prediction = self.model.predict(current_features)
        return prediction
    def calc_cci(self, high, low, close, period, add_high_to_tp, add_low_to_tp):
        """
        Calculate the Commodity Channel Index (CCI) for a given set of high, low, and close prices.
        """

        # Calculate the TP for each period
        tp = []
        for i in range(len(high)):
            if add_high_to_tp:
                tp_value = (high[i] + low[i] + close[i] + high[i]) / 4
            elif add_low_to_tp:
                tp_value = (high[i] + low[i] + close[i] + low[i]) / 4
            else:
                tp_value = (high[i] + low[i] + close[i]) / 3
            tp.append(tp_value)

        # Calculate the Simple Moving Average (SMA) of TP over the given period
        tp_sma = sum(tp[-period:]) / period

        # Calculate the Mean Deviation of TP from its SMA over the given period
        mean_deviation = sum(abs(tp[i] - tp_sma) for i in range(-period, 0)) / period
        
        # Check for mean_deviation being zero to prevent division by zero
        if mean_deviation == 0:
            return 0  # or some other value that makes sense in your context
    
        # Calculate the CCI using the formula: (TP - TP_SMA) / (0.015 * Mean Deviation)
        # The constant 0.015 normalizes the CCI so that most values fall between -100 and +100
        cci_value = (tp[-1] - tp_sma) / (0.015 * mean_deviation)

        # Return the calculated CCI value
        return cci_value


    
    def next(self):        
        """Called on each iteration of the backtest."""
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return
        current_time = self.datas[0].datetime.time(0)
        
        # if not market hours then don't process it
        if not check_market_hours(current_time):
            return
                
        # Check for peaks in OHLC data
        if self.data.high[-3] < self.data.high[-2] < self.data.high[-1] > self.data.high[0]:
            self.ohlc_peaks.append(self.data.high[-1])
            self.ohlc_peak_indexes.append(len(self.data)-1)
        
        # Check for peaks in CCI data
        if self.cci[-3] < self.cci[-2] < self.cci[-1] > self.cci[0]:
            self.cci_peaks.append(self.cci[-1])
            self.cci_peak_indexes.append(len(self.data)-1)
                
        # Check for valleys in OHLC data
        if self.data.low[-3]>self.data.low[-2] > self.data.low[-1] < self.data.low[0]:
            self.ohlc_valleys.append(self.data.low[-1])
            self.ohlc_valley_indexes.append(len(self.data)-1)

        # Check for valleys in CCI data
        if self.cci[-3]> self.cci[-2] > self.cci[-1] < self.cci[0]:
            self.cci_valleys.append(self.cci[-1])
            self.cci_valley_indexes.append(len(self.data)-1)
            
        bearish_divergence_detected=False
        bullish_divergence_detected=False   
        excess_slope = None
        # Check for bearish and bullish divergences        
        # Detect bearish divergence based on the peaks
        if len(self.ohlc_peaks) >= 2 and len(self.cci_peaks) >= 2:
            # Calculate the slope of the last two peaks for OHLC and CCI
            ohlc_slope = (self.ohlc_peaks[-1] - self.ohlc_peaks[-2]) / (self.ohlc_peak_indexes[-1] - self.ohlc_peak_indexes[-2])
            cci_slope = (self.cci_peaks[-1] - self.cci_peaks[-2]) / (self.cci_peak_indexes[-1] - self.cci_peak_indexes[-2])

            # Detect divergences based on the slopes of OHLC and CCI
            if cci_slope < ohlc_slope:
                # Bearish divergence detected
                bearish_divergence_detected = True
                self.div_marker.bearish[0] = self.data.close[0]            
        
        # Detect bullish divergences based on the valleys
        if len(self.ohlc_valleys) >= 2 and len(self.cci_valleys) >= 2:
            # Calculate the slope of the last two valleys for OHLC and CCI
            ohlc_valleys_slope = (self.ohlc_valleys[-1] - self.ohlc_valleys[-2]) / (self.ohlc_valley_indexes[-1] - self.ohlc_valley_indexes[-2])
            cci_valleys_slope = (self.cci_valleys[-1] - self.cci_valleys[-2]) / (self.cci_valley_indexes[-1] - self.cci_valley_indexes[-2])

            # Bullish divergence detected
            if cci_valleys_slope > ohlc_valleys_slope:
                bullish_divergence_detected = True
                self.div_marker.bullish[0] = self.data.close[0]                  

        # Print the divergence and excess slope information if a divergence is detected        
        if bearish_divergence_detected or bullish_divergence_detected:
            # This recalculates CCI for the current bar using the modified TP formula
            modified_cci = self.calc_cci(self.data.high.get(size=self.params.cci_period),
                                         self.data.low.get(size=self.params.cci_period),
                                         self.data.close.get(size=self.params.cci_period),
                                         self.params.cci_period,
                                         add_high_to_tp = True, add_low_to_tp = False)  # or add_high_to_tp=True
            
            # Calculate excess slope based on the type of divergence detected
            excess_slope = cci_slope - ohlc_slope if bearish_divergence_detected else cci_valleys_slope - ohlc_valleys_slope

            # Print the information
            date_time = self.data.datetime.datetime(0)
            close_price = self.data.close[0]
            divergence_type = "Bearish" if bearish_divergence_detected else "Bullish"
            self.log(f"{date_time}, {divergence_type} Divergence Detected at Close Price: {close_price} with excess slope of {excess_slope}")

        # Trading logic for SMA crossover and divergence detection
        # Update current position size
        self.current_position_size = self.position.size
        nn_prediction = self.get_nn_prediction()
        
        # Bullish strategy logic
        if not self.position:
            # If bullish crossover or bullish divergence detected
            # NN predicts higher future close and bullish signals from indicators 
            if (nn_prediction > self.datas[0].close[0]) and (self.crossover > 0 or bullish_divergence_detected):
                # If we're short 3 units, buy 6 to net long 3
                #size_to_buy = 3
                # Assign buy_order for management in notify_order
                self.buy_order = self.buy(size=None, exectype=bt.Order.Stop, trailamount=5)
            # Bearish strategy logic
            else:
                # If bearish crossover or no bullish divergence detected
                 # NN predicts lower future close and bearish signals from indicators  
                if (nn_prediction < self.datas[0].close[0]) and (self.crossover < 0 or not bearish_divergence_detected):
                    # If we're long 3 units, sell 6 to net short 3
                    #size_to_sell = 3
                    # Assign sell_order for management in notify_order
                    self.sell_order = self.sell(size=None)

        # Check if we are in the market
        if self.position:
            # print("Self position: ")
            # print(self.position)
            # Close short position if NN predicts an increase    
            if (nn_prediction > self.datas[0].close[0]) or (self.crossover > 0 or bullish_divergence_detected):
                # If we're short 3 units, buy 6 to net long 3
                #size_to_buy = 6
                # Assign buy_order for management in notify_order
                self.buy_order = self.buy(size=None, exectype=bt.Order.Stop, trailamount=5)
                self.close()
            # Bearish strategy logic
            else:
                # If bearish crossover or no bullish divergence detected
                # Close long position if NN predicts a decrease 
                if (nn_prediction < self.datas[0].close[0]) or (self.crossover < 0 or not bearish_divergence_detected):
                    # If we're long 3 units, sell 6 to net short 3
                    #size_to_sell = 6
                    # Assign sell_order for management in notify_order
                    self.sell_order = self.sell(size=None)
                    self.close()

            
# Set up backtrader engine
cerebro = bt.Cerebro(optreturn=False)  # optreturn=False to allow for detailed analysis

# Ensure correct column headers
if 'Open' not in df.columns:
    column_names = ['Open', 'High', 'Low', 'Close', 'Volume', 'Time', 'Date']
    df.columns = column_names

# Merge date and time columns and set as DataFrame index
df['DateTime'] = df['Date'].astype(str) + " " + df['Time'].astype(str)
df['DateTime'] = pd.to_datetime(df['DateTime'])
df.set_index('DateTime', inplace=True)

# Convert DataFrame to backtrader data feed
data = bt.feeds.PandasData(dataname=df)

# Add data and strategy to backtrader
cerebro.adddata(data)

# Define the range of ATR multipliers to test
atr_multipliers = [1.0, 1.5, 2.0, 2.5, 3.0]

# Add strategy to cerebro using optstrategy to test different ATR multipliers
cerebro.optstrategy(CombinedStrategy, atr_multiplier=atr_multipliers)

# Initial cash amount set to 250,000 as per requirement
cerebro.broker.set_cash(214070)
cerebro.addsizer(bt.sizers.PercentSizer, percents = 20)  # Buy 20% of equity
cerebro.broker.setcommission(commission=2.40, margin=500, mult=50)  # Futures contract

# Execute backtest and plot results
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
# Execute backtest
results = cerebro.run(maxcpus=1)  # Set maxcpus to 1 for deterministic results during optimization
print('Ending Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Analyze results
def print_results(results):
    final_results_list = []
    for run in results:
        for strategy in run:
            value = round(strategy.broker.get_value(), 2)
            PnL = round(value - 500, 2)
            atr_multiplier = strategy.params.atr_multiplier
            final_results_list.append((atr_multiplier, PnL))

    # Sort results list by PnL
    by_PnL = sorted(final_results_list, key=lambda x: x[1], reverse=True)
    print('Results: ATR Multiplier, PnL')
    for result in by_PnL:
        print(result)

print_results(results)


Starting Portfolio Value: 214070.00
1/1 [==============================] - 0s 31ms/step
2023-11-30, 2023-11-30 09:50:00, Bullish Divergence Detected at Close Price: 4554.5 with excess slope of 6.926569249365955
1/1 [==============================] - 0s 28ms/step
2023-11-30, 2023-11-30 09:55:00, Bullish Divergence Detected at Close Price: 4560.75 with excess slope of 6.926569249365955
1/1 [==============================] - 0s 27ms/step
2023-11-30, 2023-11-30 10:00:00, Bullish Divergence Detected at Close Price: 4559.5 with excess slope of 6.926569249365955
1/1 [==============================] - 0s 27ms/step
2023-11-30, 2023-11-30 10:05:00, Bullish Divergence Detected at Close Price: 4557.5 with excess slope of 6.926569249365955
1/1 [==============================] - 0s 26ms/step
2023-11-30, 2023-11-30 10:10:00, Bullish Divergence Detected at Close Price: 4559.75 with excess slope of 5.386737935085087
1/1 [==============================] - 0s 26ms/step
2023-11-30, 2023-11-30 10:15:00, Bu

1/1 [==============================] - 0s 33ms/step
2023-11-30, 2023-11-30 14:05:00, Bullish Divergence Detected at Close Price: 4555.75 with excess slope of 10.517213411459961
1/1 [==============================] - 0s 27ms/step
2023-11-30, 2023-11-30 14:20:00, Bearish Divergence Detected at Close Price: 4552.25 with excess slope of -20.8788360297857
1/1 [==============================] - 0s 29ms/step
2023-11-30, 2023-11-30 14:25:00, Bearish Divergence Detected at Close Price: 4552.75 with excess slope of -20.8788360297857
1/1 [==============================] - 0s 27ms/step
2023-11-30, 2023-11-30 14:30:00, Bearish Divergence Detected at Close Price: 4554.5 with excess slope of -20.8788360297857
1/1 [==============================] - 0s 27ms/step
2023-11-30, 2023-11-30 14:35:00, Bearish Divergence Detected at Close Price: 4556.75 with excess slope of -20.8788360297857
1/1 [==============================] - 0s 32ms/step
2023-12-01, 2023-12-01 08:30:00, Bearish Divergence Detected at Clos

1/1 [==============================] - 0s 25ms/step
2023-12-01, 2023-12-01 12:55:00, Bearish Divergence Detected at Close Price: 4594.25 with excess slope of -8.888461538478284
1/1 [==============================] - 0s 26ms/step
2023-12-01, 2023-12-01 13:00:00, Bearish Divergence Detected at Close Price: 4596.75 with excess slope of -8.888461538478284
1/1 [==============================] - 0s 35ms/step
2023-12-01, 2023-12-01 13:05:00, Bearish Divergence Detected at Close Price: 4593.0 with excess slope of -8.888461538478284
1/1 [==============================] - 0s 35ms/step
2023-12-01, 2023-12-01 13:10:00, Bearish Divergence Detected at Close Price: 4598.0 with excess slope of -8.888461538478284
1/1 [==============================] - 0s 28ms/step
2023-12-01, 2023-12-01 13:15:00, Bearish Divergence Detected at Close Price: 4596.0 with excess slope of -8.888461538478284
1/1 [==============================] - 0s 31ms/step
2023-12-01, 2023-12-01 13:20:00, Bearish Divergence Detected at Cl

1/1 [==============================] - 0s 25ms/step
2023-12-04, 2023-12-04 11:30:00, Bullish Divergence Detected at Close Price: 4575.5 with excess slope of 1.014844209284187
1/1 [==============================] - 0s 26ms/step
2023-12-04, 2023-12-04 11:35:00, Bearish Divergence Detected at Close Price: 4575.0 with excess slope of -0.9671855921793711
1/1 [==============================] - 0s 26ms/step
2023-12-04, 2023-12-04 11:40:00, Bearish Divergence Detected at Close Price: 4576.0 with excess slope of -0.9671855921793711
1/1 [==============================] - 0s 28ms/step
2023-12-04, 2023-12-04 11:45:00, Bearish Divergence Detected at Close Price: 4575.25 with excess slope of -0.9671855921793711
1/1 [==============================] - 0s 28ms/step
2023-12-04, 2023-12-04 11:50:00, Bearish Divergence Detected at Close Price: 4575.25 with excess slope of -0.9671855921793711
1/1 [==============================] - 0s 26ms/step
2023-12-04, 2023-12-04 11:55:00, Bearish Divergence Detected at

1/1 [==============================] - 0s 52ms/step
2023-12-05, 2023-12-05 10:30:00, Bearish Divergence Detected at Close Price: 4573.75 with excess slope of -0.25827067668948156
1/1 [==============================] - 0s 39ms/step
2023-12-05, 2023-12-05 10:35:00, Bearish Divergence Detected at Close Price: 4575.0 with excess slope of -2.1134050160378877
1/1 [==============================] - 0s 81ms/step
2023-12-05, 2023-12-05 10:40:00, Bearish Divergence Detected at Close Price: 4574.5 with excess slope of -2.1134050160378877
1/1 [==============================] - 0s 49ms/step
2023-12-05, 2023-12-05 10:45:00, Bearish Divergence Detected at Close Price: 4574.0 with excess slope of -2.1134050160378877
1/1 [==============================] - 0s 50ms/step
2023-12-05, 2023-12-05 10:50:00, Bearish Divergence Detected at Close Price: 4564.0 with excess slope of -2.1134050160378877
1/1 [==============================] - 0s 171ms/step
2023-12-05, 2023-12-05 10:55:00, Bearish Divergence Detected

...

2023-12-05, 2023-12-05 14:05:00, Bearish Divergence Detected at Close Price: 4575.5 with excess slope of -0.4344538359424609
1/1 [==============================] - 0s 31ms/step
2023-12-05, 2023-12-05 14:10:00, Bearish Divergence Detected at Close Price: 4575.75 with excess slope of -0.4344538359424609
1/1 [==============================] - 0s 35ms/step
2023-12-05, 2023-12-05 14:15:00, Bearish Divergence Detected at Close Price: 4576.75 with excess slope of -0.4344538359424609
1/1 [==============================] - 0s 34ms/step
2023-12-05, 2023-12-05 14:20:00, Bearish Divergence Detected at Close Price: 4577.25 with excess slope of -5.103911454978738
1/1 [==============================] - 0s 33ms/step
2023-12-05, 2023-12-05 14:25:00, Bearish Divergence Detected at Close Price: 4574.75 with excess slope of -5.103911454978738
1/1 [==============================] - 0s 33ms/step
2023-12-05, 2023-12-05 14:30:00, Bearish Divergence Detected at Close Price: 4574.75 with excess slope of -5.1039

1/1 [==============================] - 0s 43ms/step
2023-11-30, 2023-11-30 13:30:00, Bullish Divergence Detected at Close Price: 4557.0 with excess slope of 10.517213411459961
1/1 [==============================] - 0s 31ms/step
2023-11-30, 2023-11-30 13:35:00, Bullish Divergence Detected at Close Price: 4557.25 with excess slope of 10.517213411459961
1/1 [==============================] - 0s 41ms/step
2023-11-30, 2023-11-30 13:40:00, Bullish Divergence Detected at Close Price: 4560.25 with excess slope of 10.517213411459961
1/1 [==============================] - 0s 35ms/step
2023-11-30, 2023-11-30 13:45:00, Bullish Divergence Detected at Close Price: 4562.0 with excess slope of 10.517213411459961
1/1 [==============================] - 0s 37ms/step
2023-11-30, 2023-11-30 13:50:00, Bullish Divergence Detected at Close Price: 4561.75 with excess slope of 10.517213411459961
1/1 [==============================] - 0s 36ms/step
2023-11-30, 2023-11-30 13:55:00, Bullish Divergence Detected at C

1/1 [==============================] - 0s 37ms/step
2023-12-01, 2023-12-01 12:25:00, Bearish Divergence Detected at Close Price: 4596.5 with excess slope of -13.401892551892155
1/1 [==============================] - 0s 36ms/step
2023-12-01, 2023-12-01 12:30:00, Bearish Divergence Detected at Close Price: 4597.75 with excess slope of -13.401892551892155
1/1 [==============================] - 0s 30ms/step
2023-12-01, 2023-12-01 12:35:00, Bearish Divergence Detected at Close Price: 4595.5 with excess slope of -13.401892551892155
1/1 [==============================] - 0s 31ms/step
2023-12-01, 2023-12-01 12:40:00, Bearish Divergence Detected at Close Price: 4597.5 with excess slope of -13.401892551892155
1/1 [==============================] - 0s 29ms/step
2023-12-01, 2023-12-01 12:45:00, Bearish Divergence Detected at Close Price: 4595.25 with excess slope of -13.401892551892155
1/1 [==============================] - 0s 39ms/step
2023-12-01, 2023-12-01 12:50:00, Bearish Divergence Detected 

1/1 [==============================] - 0s 31ms/step
2023-12-04, 2023-12-04 10:55:00, Bullish Divergence Detected at Close Price: 4569.5 with excess slope of 28.931841967564637
1/1 [==============================] - 0s 33ms/step
2023-12-04, 2023-12-04 11:00:00, Bullish Divergence Detected at Close Price: 4569.5 with excess slope of 28.931841967564637
1/1 [==============================] - 0s 31ms/step
2023-12-04, 2023-12-04 11:05:00, Bullish Divergence Detected at Close Price: 4570.75 with excess slope of 28.931841967564637
1/1 [==============================] - 0s 31ms/step
2023-12-04, 2023-12-04 11:10:00, Bullish Divergence Detected at Close Price: 4571.25 with excess slope of 28.931841967564637
1/1 [==============================] - 0s 28ms/step
2023-12-04, 2023-12-04 11:15:00, Bullish Divergence Detected at Close Price: 4570.75 with excess slope of 1.014844209284187
1/1 [==============================] - 0s 18ms/step
2023-12-04, 2023-12-04 11:20:00, Bullish Divergence Detected at Cl

1/1 [==============================] - 0s 31ms/step
2023-12-05, 2023-12-05 10:00:00, Bullish Divergence Detected at Close Price: 4584.25 with excess slope of 8.805287675018384
1/1 [==============================] - 0s 16ms/step
2023-12-05, 2023-12-05 10:05:00, Bullish Divergence Detected at Close Price: 4581.75 with excess slope of 8.805287675018384
1/1 [==============================] - 0s 33ms/step
2023-12-05, 2023-12-05 10:10:00, Bearish Divergence Detected at Close Price: 4577.0 with excess slope of -0.25827067668948156
1/1 [==============================] - 0s 32ms/step
2023-12-05, 2023-12-05 10:15:00, Bearish Divergence Detected at Close Price: 4567.5 with excess slope of -0.25827067668948156
1/1 [==============================] - 0s 30ms/step
2023-12-05, 2023-12-05 10:20:00, Bearish Divergence Detected at Close Price: 4569.75 with excess slope of -0.25827067668948156
1/1 [==============================] - 0s 31ms/step
2023-12-05, 2023-12-05 10:25:00, Bearish Divergence Detected 